# Seattle Collisions ETL

This workbook merges weather data with the collision data.  While the collision data has a weather attribute it is categorical without supporting details.  Merging the weather is done by taking a weather file extracted from the Seattle KBFI station and merging it with the collision data.  There are date/time holes in the weather data so this needs to be handled but for the most part there is a weather reading for every collision.
<BR>
There is also the creation of a secondary weather file that is use for time series regression.  This just rolls us the weather a bit differently so it can be used with Prophet.

In [6]:
import pandas as pd
import types
import itertools
import numpy as np
from datetime import datetime, timedelta, timezone 
import time
from astral import Astral

import os
import folium
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
%matplotlib inline

In [2]:
df = pd.read_csv('Seattle_Collisions.csv', low_memory=False)
print('File downloaded')

File downloaded


In [ ]:
df.head()

In [ ]:
df.isna().sum()/len(df)

In [5]:
df.dropna(subset=['X', 'Y'],inplace=True)
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])
df['HOUR'] = df['INCDTTM'].dt.strftime('%H').astype(int)
df['DATE'] = df.INCDTTM.dt.strftime('%m/%d/%y')

In [6]:
df = df.assign(TEMPERATURE=np.NaN, WINDCHILL=np.NaN,PRECIPITATION=np.NaN,SNOWDEPTH=np.NaN,\
          VISIBILITY=np.NaN, CLOUDCOVER=np.NaN, WEATHERCONDITIONS='', SUNSET=pd.NaT, SUNRISE=pd.NaT, MOONPHASE=np.NaN)

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [11]:
weather_raw = pd.read_csv('seattle_weather-01-01-2004_06-01-2020.csv',low_memory=True,index_col=0)

In [ ]:
weather_raw.head().T

In [13]:
weather_raw.drop(['Name','Maximum Temperature', 'Minimum Temperature','Heat Index','Relative Humidity',\
                  'Wind Speed','Wind Gust'], axis=1, inplace = True)

In [14]:
weather_raw.rename(columns = {'Temperature':'TEMPERATURE', 'Wind Chill':'WINDCHILL','Precipitation':'PRECIPITATION', 'Snow Depth':'SNOWDEPTH',\
                             'Visibility':'VISIBILITY','Cloud Cover':'CLOUDCOVER','Conditions':'WEATHERCONDITIONS','Sunset':'SUNSET',\
                              'Sunrise':'SUNRISE','Moon Phase': 'MOONPHASE'}, inplace = True)

In [15]:
weather_raw['TIMESTAMP'] = pd.to_datetime(weather_raw['Date time'])
weather_raw['HOUR'] = weather_raw['TIMESTAMP'].dt.strftime('%H').astype(int)
weather_raw['DATE'] = weather_raw.TIMESTAMP.dt.strftime('%m/%d/%y')
weather_raw['KEY'] = weather_raw.apply(lambda x: (x['DATE'] + '-' + str(x['HOUR'])), axis=1)
weather_raw.set_index('KEY', inplace=True)

In [ ]:
weather_raw.head().T

In [ ]:
weather_raw.isna().sum()/len(weather_raw)

In [18]:
def find_weather_reading(df_w, date, hour):
    key = date + '-' + str(hour)
    if (df_w.index == key).any() == True:
        return df_w[(df_w.index == key)].to_dict(orient='list')
    
    while len(df_w[(df_w.DATE == date)]) == 0:
#        print('Weather gap - Date: ' + date + ' ' + str(hour))
        date2 = datetime.strptime(date, '%m/%d/%y')
        date2 = date2 + timedelta(days=1)
        date = date2.strftime('%m/%d/%y')

    while len(df_w[(df_w.DATE == date) & (df_w.HOUR == hour)]) == 0:
#        print('Weather gap - Hour: ' + date + ' ' + str(hour))
        if hour == 23: 
            hour = 0
        else:
            hour += 1
    key = date + '-' + str(hour)
    return df_w[(df_w.index == key)].to_dict(orient='list')

In [19]:
find_weather_reading(weather_raw, '03/27/13', 14)

{'Date time': ['03/27/2013 14:00:00'],
 'TEMPERATURE': [50.7],
 'WINDCHILL': [nan],
 'PRECIPITATION': [0.0],
 'SNOWDEPTH': [nan],
 'VISIBILITY': [9.9],
 'CLOUDCOVER': [0.6],
 'WEATHERCONDITIONS': ['Clear'],
 'SUNSET': ['2013-03-27T19:32:39-07:00'],
 'SUNRISE': ['2013-03-27T06:57:14-07:00'],
 'MOONPHASE': [0.5],
 'TIMESTAMP': [Timestamp('2013-03-27 14:00:00')],
 'HOUR': [14],
 'DATE': ['03/27/13']}

In [ ]:
copy_count = 1
when_to_log = 1000
for row in df.itertuples(index=True):
    try:
        i = getattr(row, 'Index')
        s = find_weather_reading(weather_raw, getattr(row, 'DATE'), getattr(row, 'HOUR'))
        df.at[i, 'TEMPERATURE'] = s['TEMPERATURE'][0]
        df.at[i, 'WINDCHILL'] = s['WINDCHILL'][0]
        df.at[i, 'PRECIPITATION'] = s['PRECIPITATION'][0]
        df.at[i, 'SNOWDEPTH'] = s['SNOWDEPTH'][0]
        df.at[i, 'VISIBILITY'] = s['VISIBILITY'][0]
        df.at[i, 'CLOUDCOVER'] = s['CLOUDCOVER'][0]
        df.at[i, 'WEATHERCONDITIONS'] = s['WEATHERCONDITIONS'][0]
        df.at[i, 'SUNSET'] = s['SUNSET'][0]
        df.at[i, 'SUNRISE'] = s['SUNRISE'][0]
        df.at[i, 'MOONPHASE'] = s['MOONPHASE'][0]
       # break
        if np.mod(copy_count, when_to_log) == 0:
            print('Status: ' + str(copy_count))
        copy_count += 1
    except Exception as e: 
        print(row)
        print(e)
        break

In [ ]:
df.head().T

In [ ]:
df.to_csv('Seattle_Collisions_Weather-1.csv')

In [21]:
a = Astral()
a.solar_depression = 'civil'
seattle_astral  = a['Seattle']

def solar_azimuth(ts):
    try:
        #ts = datetime.strptime(ts, '%Y/%m/%d')
        ts = pd.to_datetime(ts)
        return seattle_astral.solar_azimuth(ts)
    except:
        return np.nan

def solar_elevation(ts):
    try:
        return seattle_astral.solar_elevation(ts)
    except:
        return np.nan

# Generate a daily weather file - Needed for Time Series

In [24]:
weather_raw = pd.read_csv('seattle_weather-01-01-2004_06-01-2020.csv',low_memory=True,index_col=0)
weather_raw.drop(['Name','Maximum Temperature', 'Minimum Temperature','Heat Index','Relative Humidity',\
                  'Wind Speed','Wind Gust'], axis=1, inplace = True)
weather_raw.rename(columns = {'Temperature':'TEMPERATURE', 'Wind Chill':'WINDCHILL','Precipitation':'PRECIPITATION', 'Snow Depth':'SNOWDEPTH',\
                             'Visibility':'VISIBILITY','Cloud Cover':'CLOUDCOVER','Conditions':'WEATHERCONDITIONS','Sunset':'SUNSET',\
                              'Sunrise':'SUNRISE','Moon Phase': 'MOONPHASE'}, inplace = True)
weather_raw['TIMESTAMP'] = pd.to_datetime(weather_raw['Date time'])
weather_raw['HOUR'] = weather_raw['TIMESTAMP'].dt.strftime('%H').astype(int)
weather_raw['DATE'] = weather_raw.TIMESTAMP.dt.strftime('%Y-%m-%d')
weather_raw['KEY'] = weather_raw.apply(lambda x: (x['DATE'] + '-' + str(x['HOUR'])), axis=1)
weather_raw['SOLARAZIMUTH'] = weather_raw.apply(lambda x: round(solar_azimuth(x.DATE)), axis=1)
weather_raw.set_index('KEY', inplace=True)
weather_raw.drop(['Date time'], axis=1, inplace=True)

In [26]:
#weather_raw.head()
weather_raw.to_csv('Seattle_Weather_Daily.csv')